# Skyrmion in Pd/Fe/Ir(111)

In this tutorial we will reproduce Malottki et. al (Enhanced skyrmion stability due to exchange frustration
JO  - Scientific Reports 2017, https://rdcu.be/d9rHc)

In this article two types of Pd/Fe/Ir layers are studied hcp and fcc. The interaction parameters (*Table 1* in Malottki et. al) are:

### Interaction parameters

**Tab. 1:** Interaction parameters as per Malottki et. al

|         | J1    | J2    | J3    | J4   | J5   | J6   | J7   | J8    | J9    | D1   | K    |
|---------|-------|-------|-------|------|------|------|------|-------|-------|------|------|
| **hcp** | 13.66 | -0.51 | -2.88 | 0.07 | 0.55 | —    | —    | —     | —     | 1.2  | 0.8  |
| **fcc** | 14.40 | -2.48 | -2.69 | 0.52 | 0.74 | 0.28 | 0.16 | -0.57 | -0.21 | 1.0  | 0.7  |


The positive sign of the DMI indicates favoring of right rotating spin spirals and the positive sign of K indicates an easy out-of-plane magnetization direction. The coefficients are given in meV.

To set up the correct lattice, let's remind ourselved fo the bravais vectors required to specify an hcp and an fcc stacking respectively.

Like Malottki et. al. we shall assume the magnetic moment to be $3.0 \mu_B$.

### System geometry

A Pd/Fe/Ir(111) has a triangular lattice with a lattice constant $a \approx 2.72 \AA$.

So we can choose the in-plane bravais vectors
$$
\begin{aligned}
    a_1 &= a \left(1,0\right)\\
    a_2 &= a \left(\frac{1}{2},\frac{\sqrt{3}}{2}\right)
\end{aligned}
$$

In order to have fast-running simulations we will use a small system size with $32 \times 32 \times 1$ cells.

# Step 1: Fill the spirit input file

**Task:** Open the `input_fcc.cfg` and `input_hcp.cfg` files and enter the hamiltonian parameters as well as the system geometry.

**Tips**:
Make sure to adjust:
 - the number of basis cell
 - the strength and direction of the uniaxial anisotropy
 - the bravais vectors
 - the lattice constant
 - the magnetic moment
 - the Heisenberg pairs
 - the boundary conditions
 - the external magnetic field (we want to simulate the fcc system at 4T and the hcp system at 1.5T)


**Important**: Pay attention to the definition of the Hamiltonian!

Malottki et. al define
$$
\mathcal{H}_\text{exc} = \sum_{ij} J_{ij} \mathbf{m}_{i} \cdot \mathbf{m}_{j},
$$
while Spirit uses
$$
\mathcal{H}_\text{exc} = \sum_{i<j} J_{ij} \mathbf{m}_{i} \cdot \mathbf{m}_{j}.
$$
Think about how to adjust the $J_{ij}$ given by Malottki et. al. in order to be consistent with Spirit's convention! *The same applies to $D_{ij}$.*

# Step 2: Minimize the energy of the skyrmions 

After the input files have been adjusted, we can try to find a skyrmion! 

**Task**: Adjust the following code so that a suitable guess for an initial skyrmion configuration is inserted.

**Tips:** 
 - The minimization should not take longer than a few seconds. If it takes significantly longer, either your input file or your initial guess for the skyrmion is wrong.
 - All functions in spirit take the `p_state` as their first argument
 - You can use the `configuration.plus_z` and `configuration.skyrmion` functions. First create a plus_z background configuration and then insert an ansatz for the skyrmion.
 - Depending on your initial configurations, the energy minimization might not lead to a skyrmion. Make sure to experiment with the parameters (radius and phase) of the initial skyrmion guess. 


In [ ]:
from spirit import state, configuration, simulation, io
lattices = ["hcp", "fcc"]

for lattice in lattices:

    input_file = f"./input_{lattice}.cfg"
    with state.State(input_file, quiet=True) as p_state:

        # ==== Start of your code ====
        configuration.plus_z(p_state)
        configuration.skyrmion(p_state, radius=20, phase=0)
        # ==== End of your code ====

        simulation.start(p_state, simulation.METHOD_LLG, simulation.SOLVER_LBFGS_OSO)
        io.image_write(p_state, f"skyrmion_minimized_{lattice}.ovf")

### 2.1 Plot the minimized Skyrmions

Here, we plot the spin directions in a certain radius around the center of the system with `matplotlib`. If your initial guess for the skyrmion was correct you should see a large skyrmion for the `fcc` stacking parameters and a smaller one for the `hcp` stacking parameters.

**Task:** Use the `io.image_read` function to load the previously minimized and saved spin configurations.

**Tips:** If you have installed the `spirit` GUI, you may also visualize the spins by using the terminal command
```bash
spirit -f input_fcc.cfg -i skyrmion_minimized_fcc.ovf
```

In [ ]:
from spirit import state, io, geometry, system
import numpy as np
import matplotlib.pyplot as plt
from plotting import plot_spins_2d

import post_processing

fig, axes = plt.subplots(1,2)
lattices = ["hcp", "fcc"]

PLOT_RADIUS = 30

for ax, lattice in zip(axes, lattices):

    input_file = f"./input_{lattice}.cfg"
    with state.State(input_file, quiet=True) as p_state:

        # ==== Start of your code ====
        io.image_read(p_state, f"skyrmion_minimized_{lattice}.ovf")
        # ==== End of your code ====

        spins = system.get_spin_directions(p_state)
        positions = geometry.get_positions(p_state)
        center = np.mean(positions, axis=0)

        mask = np.linalg.norm(positions - center, axis=1) < PLOT_RADIUS

        ax.set_title(lattice)
        plot_spins_2d(
            ax, positions[mask], spins[mask], angles="xy", scale_units="xy", scale=0.2
        )

        radius = post_processing.skyrmion_radius(positions, spins, center)
        print(f"{radius = }")

fig.tight_layout()

# Save a plot to show your parents :)
fig.savefig("minimized_skyrmions.png", dpi=300)

# Step 3: Find the energy barriers for the skyrmion collapse

We are now equipped to find the energy barriers for the collapse of Skrymions to the ferromagnetic ground state (+z)

In [ ]:
from spirit import state, io, simulation, io, parameters, chain, transition

lattices = ["hcp", "fcc"]

N_IMAGES = 12

for ax, lattice in zip(axes, lattices):
    input_file = f"./input_{lattice}.cfg"

    with state.State(input_file, quiet=True) as p_state:
        io.image_read(p_state, f"skyrmion_minimized_{lattice}.ovf")
        chain.set_length(p_state, N_IMAGES)
        configuration.plus_z(p_state, idx_image=N_IMAGES - 1)
        transition.homogeneous(p_state, idx_1=0, idx_2=N_IMAGES - 1)

        io.chain_write(p_state, f"chain_initial_{lattice}.ovf")
        simulation.start(
            p_state, simulation.METHOD_GNEB, simulation.SOLVER_VP, n_iterations=500
        )
        parameters.gneb.set_image_type_automatically(p_state)
        simulation.start(
            p_state, simulation.METHOD_GNEB, simulation.SOLVER_VP, n_iterations=1000
        )
        io.chain_write(p_state, f"chain_final_{lattice}.ovf")

        energy_images = chain.get_energy(p_state)
        energy_barrier  = np.max(energy_images) - energy_images[0]

        print(f"{energy_barrier = }")

